### Performance Metrics

## Success Indicators

You have now been asked to discover what key performance indicators (KPIs) will determine the success of the new design? Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.

- **Completion Rate:** The proportion of users who reach the final ‘confirm’ step.
- **Time Spent on Each Step:** The average duration users spend on each step.
- **Error Rates:** If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

## Redesign Outcome

Based on the chosen KPIs, how does the new design’s performance compare to the old one?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# import reusable functions from utils directory
import sys
sys.path.append('../../utils')
import functions

In [3]:
combined_cleaned_df = pd.read_csv('../../data/raw/combined_cleaned_data.csv')
# combined_cleaned_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/combined_cleaned_data.csv'

In [ ]:
combined_cleaned_df['date_time'] = pd.to_datetime(combined_cleaned_df['date_time'], errors='coerce')

# check if any dates couldn't be converted (i.e., they are NaT)
inconsistent_dates = combined_cleaned_df['date_time'].isna().sum()

# if inconsistent_dates > 0, then there are invalid or mismatched date formats
if inconsistent_dates > 0:
    print(f'There are {inconsistent_dates} inconsistent or invalid date formats in the column.')
else:
    print('All dates in the column have the same format.')

In [ ]:
kpi_df = combined_cleaned_df.copy()
kpi_df = kpi_df.drop(columns=['visitor_id', 'visit_id'])

In [ ]:
# change the naming of the steps to numeric values to ensure order
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

kpi_df['process_step'] = kpi_df['process_step'].map(replacement_dict_steps)

# **Completion Rate**

In [ ]:
def find_completion_rate(df):    
    clients_finished = kpi_df[kpi_df['process_step'] == 4] # filter rows where process_step is 'Finish'
    total_unique_clients = kpi_df['client_id'].nunique()
    unique_clients_finished = clients_finished['client_id'].nunique() # find unique client ids that finished
    print(f'Clients who finished the process: {unique_clients_finished} out ouf {total_unique_clients}.')

find_completion_rate(kpi_df)

**To-do's**:
- [x] calculate percentage
- [x] visual representation?

In [ ]:
clients_finished = kpi_df[kpi_df['process_step'] == 4]['client_id'].nunique()
total_clients = kpi_df['client_id'].nunique()
clients_not_finished = total_clients - clients_finished

completion_rate = (clients_finished / total_clients) * 100

# pie chart
plt.figure(figsize=(8, 8))
plt.pie(
    [clients_finished, clients_not_finished],
    labels=['Finished', 'Not Finished'],
    autopct='%1.1f%%',
    colors=['lightgreen', 'salmon'],
    startangle=140
)
plt.title('Completion Rate')
plt.show()

# bar chart
plt.figure(figsize=(8, 8))
bars = plt.bar(['Total Clients', 'Completed Clients'], [total_clients, clients_finished], color=['skyblue', 'green'])

# add percentage annotation on top
plt.text(1, clients_finished + 1, f'{completion_rate:.1f}%', ha='center', va='bottom', fontsize=12, color='black')

plt.title('Total Clients vs Completed Clients')
plt.ylabel('Number of Clients')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# **Time spent on each step**

In [ ]:
# sort df by client_id and date_time
kpi_df = kpi_df.sort_values(by=['client_id', 'date_time'])

# calculate time difference between steps per client
kpi_df['time_diff'] = kpi_df.groupby('client_id')['date_time'].diff()

# drop first row per client (contains no information)
kpi_df = kpi_df.dropna(subset=['time_diff'])

In [ ]:
# calculate average time spent on each step
avg_time_per_step = kpi_df.groupby('process_step')['time_diff'].mean()

# convert to total seconds
avg_time_in_seconds = avg_time_per_step.dt.total_seconds()

# round up to the nearest second to remove decimals
avg_time_in_seconds_rounded = np.ceil(avg_time_in_seconds).astype(int)

# convert back to timedelta
avg_time_per_step_rounded = pd.to_timedelta(avg_time_in_seconds_rounded, unit='s')

# re-map process_step index for readability
# avg_time_per_step_rounded.index = avg_time_per_step_rounded.index.map({0: 'Start', 1: 'Step 1', 2: 'Step 2', 3: 'Step 3', 4: 'Finish'})

print('Average time spent on each step:\n')
print(avg_time_per_step_rounded)

**To-do's**:
- [x] calculate percentage
- [x] visual representation?
- [ ] Convert to seconds?

In [ ]:
avg_time_in_seconds_rounded.index = avg_time_in_seconds_rounded.index.map({0: 'Start', 1: 'Step 1', 2: 'Step 2', 3: 'Step 3', 4: 'Finish'})

# horizontal bar chart
plt.figure(figsize=(10, 6))
avg_time_in_seconds_rounded.sort_values(ascending=True).plot(kind='barh', color='cornflowerblue')
plt.title('Average Time Spent on Each Step')
plt.xlabel('Time (in seconds)')
plt.ylabel('Process')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# **Error Rates**
_(Constanza's code)_

In [ ]:
df = pd.read_csv('../../data/raw/combined_cleaned_data.csv')
# df

In [ ]:
# drop columns
df = df.drop(columns=['visitor_id', 'visit_id'])

# change steps to numeric values to ensure order
replacement_dict_steps = {
    'start' : 0,
    'step_1' : 1,
    'step_2' : 2,
    'step_3' : 3,
    'confirm' : 4
}

df['process_step'] = df['process_step'].map(replacement_dict_steps)

In [ ]:
# sort df by client_id and date_time
df = df.sort_values(by=['client_id', 'date_time'])

In [ ]:
# Detect backward steps (errors) for all clients
df['backward_step'] = df.groupby('client_id')['process_step'].diff() < 0

# Count the number of unique clients who experienced at least one error
clients_with_errors = df[df['backward_step']]['client_id'].nunique()

# Count the total number of errors for each client
errors_per_client = df[df['backward_step']].groupby('client_id').size()

# Total number of errors across all clients
total_errors = df['backward_step'].sum()

# Output results
print(f'Number of clients with at least one error: {clients_with_errors}')
print('Errors per client:')
print(errors_per_client)
print(f'Total number of errors across all clients: {total_errors}')

In [ ]:
# add column 'error_count'
df['error_count'] = df['client_id'].map(errors_per_client).fillna(0).astype(int)

In [ ]:
# # Crear un DataFrame con los resultados
# results_df = errors_per_client.reset_index()
# results_df.columns = ['client_id', 'number_of_errors']

# # Guardar el DataFrame en un archivo CSV
# results_df.to_csv('errors_per_client.csv', index=False)

# # Mensaje de confirmación
# print('El archivo 'errors_per_client.csv' se ha guardado correctamente.')

In [ ]:
# # Bar Chart for Errors per Client
# plt.figure(figsize=(10, 6))
# errors_per_client.sort_values(ascending=False).plot(kind='bar', color='skyblue')
# plt.title('Total Errors Per Client')
# plt.xlabel('Client ID')
# plt.ylabel('Number of Errors')
# plt.xticks(rotation=45)
# plt.grid(axis='y')
# plt.tight_layout()
# plt.show()

# # Pie Chart for Proportion of Clients with Errors
# clients_without_errors = df['client_id'].nunique() - clients_with_errors
# plt.figure(figsize=(8, 8))
# plt.pie(
#     [clients_with_errors, clients_without_errors],
#     labels=['With Errors', 'Without Errors'],
#     autopct='%1.1f%%',
#     colors=['salmon', 'lightgreen'],
#     startangle=140
# )
# plt.title('Proportion of Clients with Errors')
# plt.show()

# **Redesign Outcome**
Based on the chosen KPIs, how does the new design’s performance compare to the old one?

For the project requirements and instructions for today’s tasks in full, please refer to the project brief. However, in order to keep on track you may refer to the daily goals outlined below:

By the end of day, we recommend you have:

Reviewed KPI and Metrics material.
Discovered what key performance indicators (KPIs) will determine the success of the new design
Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.
Evaluated how the new design’s performance compare to the old one, given the chosen KPIs (completion rate, time spent on each step and error rates).

Steps:
- [x] import `merged_final_demo_final_experiment_clients_df`
- [x] merge with `kpi_df`
- [x] add column of errors per client
- [x] clean data if needed

In [ ]:
merged_final_demo_final_experiment_clients_df = pd.read_csv('../../data/clean/merged_final_demo_final_experiment_clients_df.csv')

In [ ]:
merged_df = pd.merge(kpi_df, merged_final_demo_final_experiment_clients_df, on='client_id', how='outer')

In [ ]:
# match datatypes in dfs before merging on `date_time` column to prevent errors
merged_df['date_time'] = pd.to_datetime(merged_df['date_time'], errors='coerce')
df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')

In [ ]:
# select columns to merge
df_selected = df[['client_id', 'date_time', 'backward_step', 'error_count']]
merged_kpi_df = pd.merge(merged_df, df_selected, on=['client_id', 'date_time'], how='left')

In [ ]:
# merged_kpi_df: 5682251 rows, 15 columns - before matching date_time
# merged_kpi_df: 631469 rows × 15 columns - after matching date and time

In [ ]:
merged_kpi_df.rename(columns={'backward_step': 'stepped_back'}, inplace=True)

In [ ]:
merged_kpi_df.drop(columns=['client_tenure_months', 'num_accounts', 'calls_last_6_months', 'logons_last_6_months', 'balance'], inplace=True)

In [ ]:
functions.floats_to_ints(merged_kpi_df, 'process_step')
functions.floats_to_ints(merged_kpi_df, 'client_tenure_years')
functions.floats_to_ints(merged_kpi_df, 'client_age')
functions.floats_to_ints(merged_kpi_df, 'error_count')

In [ ]:
# convert NaN values in 'variation' to 'Unknown'
merged_kpi_df['variation'] = merged_kpi_df['variation'].fillna('Unknown')

In [ ]:
functions.handle_unique_gender_values(merged_kpi_df)

### Evaluate how the new design’s performance compare to the old one, given the chosen KPIs (completion rate, time spent on each step and error rates).



In [ ]:
# split in dfs, for Variation Control and Variation Test
df_test_kpi = merged_kpi_df[merged_kpi_df['variation'] == 'Test']
df_control_kpi = merged_kpi_df[merged_kpi_df['variation'] == 'Control']
df_unknown_kpi = merged_kpi_df[merged_kpi_df['variation'] == 'Unknown']

In [ ]:
# drop empty rows
df_test_kpi = df_test_kpi.dropna()
df_control_kpi = df_control_kpi.dropna()
df_unknown_kpi = df_unknown_kpi.dropna()

## Completion Rates

In [ ]:
print('\nCompletion rate for Control:')
functions.find_completion_rate(df_control_kpi)

In [ ]:
print('\nCompletion rate for Test:')
functions.find_completion_rate(df_test_kpi)

## Time spent on each step

In [ ]:
functions.calculate_time_per_step(df_test_kpi)

In [ ]:
functions.calculate_time_per_step(df_control_kpi)

## Error Rate

In [ ]:
functions.find_error_rate(df_test_kpi)

In [ ]:
functions.find_error_rate(df_control_kpi)

In [ ]:
functions.find_error_rate(df_unknown_kpi)

# Storing values

In [ ]:
# store the results for test df
test_unique_finished, test_total_clients, test_completion_rate = functions.find_completion_rate(df_test_kpi)
test_avg_time_per_step = functions.calculate_time_per_step(df_test_kpi)
test_clients_with_errors, test_errors_per_client, test_total_errors = functions.find_error_rate(df_test_kpi)

# store the results for control df
control_unique_finished, control_total_clients, control_completion_rate = functions.find_completion_rate(df_control_kpi)
control_avg_time_per_step = functions.calculate_time_per_step(df_control_kpi)
control_clients_with_errors, control_errors_per_client, control_total_errors = functions.find_error_rate(df_control_kpi)

# store the results for unknown df
unknown_finished, unknown_total_clients, unknown_completion_rate = functions.find_completion_rate(df_unknown_kpi)
unknown_avg_time_per_step = functions.calculate_time_per_step(df_unknown_kpi)
unknown_clients_with_errors, unknown_errors_per_client, unknown_total_errors = functions.find_error_rate(df_unknown_kpi)

## **Testing Plots**

In [ ]:
def plot_completion_rate(test_completion_rate, control_completion_rate):
    completion_rate_data = {
        'Version': ['Test', 'Control'],
        'Completion Rate': [test_completion_rate, control_completion_rate]
    }
    completion_rate_df = pd.DataFrame(completion_rate_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Version', y='Completion Rate', data=completion_rate_df)
    
    ax.patches[0].set_facecolor('lightblue')
    ax.patches[1].set_facecolor('salmon') 
    
    plt.title('Completion Rate (Test vs Control)', fontsize=16)
    plt.ylabel('Completion Rate (%)')
    plt.show()

In [ ]:
def plot_avg_time_per_step(test_avg_time_per_step, control_avg_time_per_step):
    test_avg_time_seconds = test_avg_time_per_step.dt.total_seconds().round() # convert to seconds
    control_avg_time_seconds = control_avg_time_per_step.dt.total_seconds().round()

    time_data = {
        'Process Step': test_avg_time_per_step.index,
        'Test': test_avg_time_seconds.values,
        'Control': control_avg_time_seconds.values
    }
    time_df = pd.DataFrame(time_data)
    
    time_df.set_index('Process Step').plot(kind='bar', figsize=(10, 6), width=0.8, color=['lightblue', 'salmon'])
    plt.title('Average Time per Step (Test vs Control)', fontsize=16)
    plt.ylabel('Average Time (in seconds)')
    plt.xlabel('Step within the process')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_error_rate(test_clients_with_errors, control_clients_with_errors, test_total_errors, control_total_errors):
    clients_error_data = {
        'Version': ['Test', 'Control'],
        'Clients': [test_clients_with_errors, control_clients_with_errors]
    }
    clients_error_df = pd.DataFrame(clients_error_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Version', y='Clients', data=clients_error_df)
    
    ax.patches[0].set_facecolor('cadetblue')
    ax.patches[1].set_facecolor('lightcoral')
    
    plt.title('Client Error Count (Test vs Control)', fontsize=16)
    plt.show()

    total_errors_data = {
        'Version': ['Test', 'Control'],
        'Total Errors': [test_total_errors, control_total_errors]
    }
    total_errors_df = pd.DataFrame(total_errors_data)
    
    plt.figure(figsize=(8, 6))
    ax = sns.barplot(x='Version', y='Total Errors', data=total_errors_df)
    
    ax.patches[0].set_facecolor('cadetblue')
    ax.patches[1].set_facecolor('lightcoral')
    
    plt.title('Total Error Count (Test vs Control)', fontsize=16)
    plt.show()

In [ ]:
plot_completion_rate(test_completion_rate, control_completion_rate)

In [ ]:
plot_avg_time_per_step(test_avg_time_per_step, control_avg_time_per_step)

In [ ]:
plot_error_rate(test_clients_with_errors, control_clients_with_errors, test_total_errors, control_total_errors)

## **More Plots!**

In [ ]:
# **Completion Rate Comparison - Pie Chart**

def plot_completion_rate_pie(test_completion_rate, control_completion_rate, unknown_completion_rate):
    completion_rate_data = {
        'Version': ['Test', 'Control', 'Unknown'],
        'Completion Rate': [test_completion_rate, control_completion_rate, unknown_completion_rate]
    }
    completion_rate_df = pd.DataFrame(completion_rate_data)
    
    # Pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(completion_rate_df['Completion Rate'], labels=completion_rate_df['Version'], autopct='%1.1f%%', startangle=90, colors=['lightblue', 'salmon', 'gray'])
    plt.title('Completion Rate (Test vs Control)')
    plt.show()

In [ ]:
# **Average Time per Step - Boxplot**

def plot_avg_time_per_step_boxplot(test_avg_time_per_step, control_avg_time_per_step):
    test_avg_time_seconds = test_avg_time_per_step.dt.total_seconds().round()  # Use .dt.total_seconds() for Series
    control_avg_time_seconds = control_avg_time_per_step.dt.total_seconds().round()  # Use .dt.total_seconds() for Series
    
    time_df = pd.DataFrame({
        'Process Step': ['Start', 'Step 1', 'Step 2', 'Step 3', 'Finish'],
        'Test': test_avg_time_seconds,
        'Control': control_avg_time_seconds
    })

    time_df = time_df.melt(id_vars='Process Step', value_vars=['Test', 'Control'], var_name='Version', value_name='Time (in seconds)')

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Process Step', y='Time (in seconds)', hue='Version', data=time_df)
    plt.title('Average Time per Step (Test vs Control)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# **Error Rate - Heatmap of Errors per Client**

def plot_error_heatmap(test_errors_per_client, control_errors_per_client):
    error_data = {
        'Test': test_errors_per_client,
        'Control': control_errors_per_client
    }
    error_df = pd.DataFrame(error_data).T  # transpose to get rows as groups
    
    plt.figure(figsize=(8, 6))
    # Use 'f' for floating-point numbers, or use '.2f' for two decimal places
    sns.heatmap(error_df, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, cbar=True)
    plt.title('Error Count Heatmap (Test vs Control)')
    plt.show()

In [ ]:
plot_completion_rate_pie(test_completion_rate, control_completion_rate, unknown_completion_rate)

In [ ]:
plot_avg_time_per_step_boxplot(test_avg_time_per_step, control_avg_time_per_step)

In [ ]:
plot_error_heatmap(test_errors_per_client, control_errors_per_client)